# FEATURE ENGINEERING, FEATURE SELECTION, MODEL BUILDING

In [1]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer, StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score, roc_auc_score
from sklearn.metrics import plot_confusion_matrix, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier 
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE,SelectKBest
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.feature_selection import f_classif, chi2

In [4]:
# Loading the Data
df_test = pd.read_csv('C:\\Users\\nidhi\\Desktop\Hackton\GREYATOM\Travel_Insurance\Data\train.csv')
df_train = pd.read_csv('C:\Users\nidhi\Desktop\Hackton\GREYATOM\Travel_Insurance\Data\train.csv')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-4-e420ddd3308c>, line 2)

# Feature Engineering

## Numerical Data

### Handling Outliers

In [ ]:
#Box Plot before outlier treatment
plt.figure(figsize=(6,6))
ax = sns.boxplot(y=df_train["Age"])

In [ ]:
plt.figure(figsize=(6,6))
ax = sns.boxplot(y=df_train["Duration"])

- Duration: Min = -2 Max = 4881 But can insurance company giving insurance for 4881 days? <br>
Insurance company do not give insurance over 360 days. Further extension will be provided for 180 days.
we are considering 540 days as a limit.

- Age. Min = 0 Max = 118 age 0 can be possible but 118 too old however we need to check too old people gets insurance or not. So,Insurance Companies do not give insurance over an age of 85.However there are few who provide insurance upto age of 99. so we treat the values above 100.

In [ ]:
df_train[df_train['Duration'] > 540 ].shape
df_train[df_train['Age'] > 99 ].shape


In [ ]:
# Write a function for data preprocessing
def age_convert(age):
    result = ''
    if(age <= 21):
        result = 'Child'
    elif(age <= 50):
        result = 'Adult'
    else:
        result = 'Senior'
    return result


def pre_processing(train_data):
    train_data['Age Group'] = train_data['Age'].map(lambda x: age_convert(x))
    train_data['Duration'][train_data['Duration'] < 0] = train_data['Duration'].median()
    train_data['Duration'][train_data['Duration'] > 540] = 540
    train_data['Age'][train_data['Age'] > 99] = train_data[train_data['Age Group'] == 'Senior']['Age'].mean()
  

In [ ]:
pre_processing(df_train)

In [ ]:
#Box Plot after outlier treatment
plt.figure(figsize=(6,6))
ax = sns.boxplot(y = df_train["Age"])

In [ ]:
plt.figure(figsize=(6,6))
ax = sns.boxplot(y = df_train["Duration"])

In [ ]:
df_train.head(2)

In [ ]:
df_test.head(2)

## Categorical Data


#### One Hot Encoding

In [ ]:
# Categorical Columns to drop 
cat_col = ['Agency', 'Agency Type', 'Distribution Channel', 'Product Name', 'Destination']

In [ ]:
X_cat = df_train[cat_col]
X_cat.head(2)
X_cat_one_hot_train = pd.get_dummies(X_cat, drop_first=True)

#Merge With original df
cat_proc_columns = X_cat_one_hot_train.columns
for col in cat_proc_columns:
    df_train[col] = X_cat_one_hot_train[col] 
    
df_train.head(2)

In [ ]:
X_cat = df_test[cat_col]
X_cat_one_hot = pd.get_dummies(X_cat, drop_first=True)

#Merge With original df
cat_proc_columns = X_cat_one_hot.columns
for col in cat_proc_columns:
    df_test[col] = X_cat_one_hot[col] 
    
df_test.head(2)



In [ ]:
df_test = df_test.drop(columns = cat_col)
df_train = df_train.drop(columns = cat_col)

In [ ]:
df_test = df_test.drop(columns=['ID'])
df_test.head(2)

In [ ]:
df_train = df_train.drop(columns=['ID'])
df_train.head(2)

# Feature Selection

## Numerical Data [Co-relation ]


In [ ]:
updated_data = df_train
updated_data_test = df_test

In [ ]:
# checking crelation in numerical column
#num_df=updated_data.select_dtypes(include=['number']).copy()
df_num = df_train[['Duration', 'Age', 'Net Sales', 'Commision (in value)', 'Claim']]

In [ ]:
#ploting heatmap to check corelation in numerical feature in data.
fig = plt.figure(figsize=(10, 7))
sns.heatmap(df_num.corr(), annot=True)
plt.title('Heatmap for detecting multicollinearity', fontsize=16, color='navy')
plt.show()

- Highest Co related features are Net Sales vs Commission. Further Process or drop.



### Numerical Input, Categorical Output ==> ANOVA

In [ ]:
df_num = df_train[['Duration', 'Age', 'Net Sales', 'Commision (in value)', 'Claim']]
df_num = df_num[df_num.Duration > 0]
X = df_num.iloc[:,:-1]
y = df_num.iloc[:,-1]

# Apply Anova and fit the logistic model on train data use df dataset
nof_list   = [1, 2, 3, 4]
high_score = 0
nof        = 0

for n in nof_list:
    test = SelectKBest(score_func = f_classif , k = n)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)
    X_train = test.fit_transform(X_train,y_train)
    X_test = test.transform(X_test)
    model = RandomForestClassifier()
    model.fit(X_train,y_train)
    
    print("For no of features=",n,", score=", model.score(X_test,y_test))
    
    if model.score(X_test,y_test) > high_score:
        high_score = model.score(X_test, y_test)
        model_ = model
        nof=n 
    print("High Score is:",high_score, "with features=", nof)

- High Score is for 1. Not changing much can go ahead with 4 features 

In [ ]:
feat_importances.nlargest(10)

# Model Selection

In [ ]:
y = updated_data['Claim']
X = updated_data.drop(columns=['Claim'])

In [ ]:
X = X.drop(columns='Age Group')
X.head(3)

## Balanced Dataset [SMOTE]


In [ ]:
smote = SMOTE(random_state=7)
X_SMOTE, y_SMOTE = smote.fit_resample(X, y)
X_train_smote, X_test_smote, y_train_smote, y_test_smote = train_test_split(X_SMOTE, y_SMOTE, test_size=0.10, random_state=42)

## Balanced Dataset [SMOTE Tomek]

In [ ]:
tomek = SMOTETomek(random_state=42)
X_SMOTE_TK, y_SMOTE_TK = smote.fit_resample(X, y)
X_train_smotetk, X_test_smotetk, y_train_smotetk, y_test_smotetk = train_test_split(X_SMOTE_TK, y_SMOTE_TK, test_size=0.10, random_state=42)

## Models


In [ ]:
models = {'random_forest':RandomForestClassifier(min_samples_split= 2, min_samples_leaf= 1,\
    max_features= 'sqrt', max_depth= 560,criterion= 'entropy'), 
          'logistic_reg':LogisticRegression(), 
          'XGB':XGBClassifier(), 
          'DT': DecisionTreeClassifier()}

In [ ]:
def process_model(models, X_train, y_train):
    for name, model in models.items():
        score = cross_val_score(model, X_train, y_train, cv=StratifiedKFold(shuffle=True),\
                                scoring='recall', n_jobs=-1)
        print(f'{name} Precision score : {np.mean(score)}')
        model.fit(X_train, y_train)
        y_pred = model.predict(updated_data_test1)
        y_pred1 = model.predict(X_test)
        print("Confusion Matrix: \n",confusion_matrix(y_test, y_pred1))
        print("=="*20)
        print("Classification Report: \n",classification_report(y_test,y_pred1))



# Random Forest Classification

In [ ]:
y = df_train['Claim']
X = df_train.drop(columns=['Claim'])


plt.figure(figsize=(5, 5))
# Build a forest and compute the feature importances
rff = RandomForestClassifier(max_depth=4, max_features='sqrt', min_samples_leaf=5,
                       n_estimators=50, random_state=9)

rff.fit(X, y)

#plot graph of feature importances for better visualization
feat_importances = pd.Series(forest.feature_importances_, index=X.columns)
feat_importances.nlargest(10).sort_values(ascending=True).plot(kind='barh')
plt.show()

In [ ]:
y = df_train['Claim']
X = df_train.drop(columns=['Claim'])

column_names = ['Duration', 'Net Sales', 'Age', 'Commision (in value)', 'Agency_C2B', 'Destination_SINGAPORE', \
               'Agency Type_Travel Agency']
X = X[column_names]
updated_data_test = updated_data_test[column_names]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.3)
scaler = StandardScaler()
#scaler  = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)



plt.figure(figsize=(10, 10))
# Build a forest and compute the feature importances
rff = RandomForestClassifier(max_depth=4, max_features='sqrt', min_samples_leaf=5,
                       n_estimators=50, random_state=9)

rff.fit(X_train, y_train)



y_pred1 = rff.predict(X_test)
print("Confusion Matrix: \n",confusion_matrix(y_test, y_pred1))
print("=="*20)
print("Classification Report: \n",classification_report(y_test,y_pred1))

In [ ]:
df_test = pd.read_csv('./file/test.csv')

In [ ]:
dt_ID = df_test['ID']

In [ ]:
predictions = rff.predict(df_test)
len(list(predictions))